In [6]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3



# Working with SQlite Databases using Python and Pandas


<!-- PELICAN_BEGIN_SUMMARY -->

Working with SQLite databases (can be imported from Excel) to visiual/interactive financial analysis 

<!-- PELICAN_END_SUMMARY -->


## Sample is the 5 years of Dow30 stocks
- Query Databases Row in Python
- Mapping Dow30 Stocks over 10 years period 
- Reading results into a pandas DataFrame
- Mapping Dow30 Stocks Yield Ratio
- Modifying database rows
- Inserting rows with Python
- Passing parameters into a query
- Updating rows
- Deleting rows
- Creating tables
- Creating tables with pandas
- Altering tables with Pandas
- Visualize chart
- Export out the file to Excel

